<style>
.arch-table {
  border-collapse: collapse;
  width: 100%;
  font-family: Arial, sans-serif;
  font-size: 14px;
}

.arch-table th {
  background-color: #1f4fd8;
  color: #ffffff;
  padding: 10px;
  border: 1px solid #333;
  text-align: center;
}

.arch-table td {
  padding: 10px;
  border: 1px solid #333;
  vertical-align: top;
}

.arch-table tr:nth-child(even) {
  background-color: #f4f6fb;
}

.arch-table .v1 {
  background-color: #e3f2fd;
}

.arch-table .v2 {
  background-color: #e8f5e9;
}

.arch-title {
  font-family: Arial, sans-serif;
  font-size: 18px;
  font-weight: bold;
  margin-bottom: 10px;
  color: #1f4fd8;
}

.arch-subtitle {
  font-size: 13px;
  margin-bottom: 16px;
  color: #333;
}
</style>

<div class="arch-title">
SIGNet Architecture Comparison (v1 vs v2)
</div>

<div class="arch-subtitle">
Comparison of baseline and enhanced digital-twin architectures used for driver identification.
</div>

<table class="arch-table">
  <tr>
    <th>Aspect</th>
    <th>SIGNet-v1 (Baseline)</th>
    <th>SIGNet-v2 (Enhanced)</th>
  </tr>

  <tr>
    <td><b>Design Objective</b></td>
    <td class="v1">
      Proof-of-concept conditional VAE for driver-wise digital twin generation.
    </td>
    <td class="v2">
      Robust, deployment-ready digital twin with improved stability, separability, and generalization.
    </td>
  </tr>

  <tr>
    <td><b>Encoder Depth</b></td>
    <td class="v1">
      Single hidden layer (shallow encoder).
    </td>
    <td class="v2">
      Multi-layer encoder with increased representational capacity.
    </td>
  </tr>

  <tr>
    <td><b>Decoder Depth</b></td>
    <td class="v1">
      Symmetric shallow decoder.
    </td>
    <td class="v2">
      Deeper decoder enabling higher-fidelity signal reconstruction and generation.
    </td>
  </tr>

  <tr>
    <td><b>Latent Dimension</b></td>
    <td class="v1">
      Low-dimensional latent space (compact but limited separability).
    </td>
    <td class="v2">
      Higher-capacity latent space enabling improved inter-driver discrimination.
    </td>
  </tr>

  <tr>
    <td><b>Stabilization Techniques</b></td>
    <td class="v1">
      None (plain fully-connected layers).
    </td>
    <td class="v2">
      Batch normalization and deeper nonlinear transformations.
    </td>
  </tr>

  <tr>
    <td><b>Loss Function</b></td>
    <td class="v1">
      Standard VAE loss (Reconstruction + KL divergence).
    </td>
    <td class="v2">
      Weighted VAE loss (β-VAE style) for improved latent regularization.
    </td>
  </tr>

  <tr>
    <td><b>Training Strategy</b></td>
    <td class="v1">
      Fixed learning rate, early stopping based on validation loss.
    </td>
    <td class="v2">
      Optimized learning rate, extended patience, and stability-oriented early stopping.
    </td>
  </tr>

  <tr>
    <td><b>Latent Consistency Metrics</b></td>
    <td class="v1">
      Limited intra-class similarity metrics.
    </td>
    <td class="v2">
      Full IEEE-grade suite: cosine similarity, Euclidean spread, variance trace, coefficient of variation.
    </td>
  </tr>

  <tr>
    <td><b>Downstream Usage</b></td>
    <td class="v1">
      Latent features used for IDInferNet and DT-GDIN (baseline performance).
    </td>
    <td class="v2">
      Stronger latent embeddings yielding improved classification accuracy and robustness.
    </td>
  </tr>

  <tr>
    <td><b>Generalization Capability</b></td>
    <td class="v1">
      Moderate — sensitive to noise and distribution shifts.
    </td>
    <td class="v2">
      High — improved robustness for synthetic generation and edge inference.
    </td>
  </tr>

  </tr>

</table>


download.svg

download (2).svg

IDInferNet-v2 Architecture (Latent Classifier)
download (3).svg

DT-GDIN-v2 Architecture (Digital-Twin Generated Classifier)
download (4).svg

In [6]:
# ============================================================
# CVAE_Digital_Twin-v2.ipynb
# SIGNet-v2 + IDInferNet-v2 + DT-GDIN-v2
# IEEE Transactions–Grade | FULL | REPRODUCIBLE
# ============================================================

!pip -q install torch torchvision torchaudio thop seaborn scikit-learn openpyxl

# ============================================================
# 1. IMPORTS & GLOBAL CONFIG
# ============================================================

import os, time, copy, json, math
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, classification_report,
    confusion_matrix, roc_curve, auc
)
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from scipy.spatial.distance import euclidean
from thop import profile

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"⚙️ Using device: {device}")

# ============================================================
# 2. DATA & DIRECTORY SETUP
# ============================================================

from google.colab import drive
drive.mount("/content/drive")

DATA_ROOT = "/content/drive/MyDrive/DT_Driver_Wise_Data"
RES_ROOT  = f"{DATA_ROOT}/SIGNetV2_Results"

DIRS = {
    "fig": f"{RES_ROOT}/Figures",
    "mdl": f"{RES_ROOT}/Models",
    "xls": f"{RES_ROOT}/Excel",
}
for d in DIRS.values():
    os.makedirs(d, exist_ok=True)

DRIVERS = ["B", "D", "F"]

FEATURES = [
    "Long_Term_Fuel_Trim_Bank1",
    "Engine_coolant_temperature.1",
    "Activation_of_Air_compressor",
    "Torque_of_friction",
    "Engine_soacking_time",
    "Intake_air_pressure",
]
TIME_COL = "Time(s)"

# ============================================================
# 3. DATA LOADING
# ============================================================

dfs = []
for drv in DRIVERS:
    for split in ["Train", "Valid"]:
        p = f"{DATA_ROOT}/Driver_{drv}_{split}.csv"
        if os.path.exists(p):
            df = pd.read_csv(p)
            df["Driver"] = drv
            df["Split"] = split
            dfs.append(df)

df_all = pd.concat(dfs, ignore_index=True)
df_all = df_all.dropna(subset=FEATURES + [TIME_COL])

# ============================================================
# 4. PREPROCESSING
# ============================================================

sc_x = StandardScaler()
sc_t = StandardScaler()
enc  = OneHotEncoder(sparse_output=False)

X = sc_x.fit_transform(df_all[FEATURES])
T = sc_t.fit_transform(df_all[[TIME_COL]])
D = enc.fit_transform(df_all[["Driver"]])
C = np.concatenate([T, D], axis=1)

X = torch.tensor(X, dtype=torch.float32).to(device)
C = torch.tensor(C, dtype=torch.float32).to(device)

mask_tr = df_all["Split"] == "Train"
mask_va = df_all["Split"] == "Valid"

X_tr, C_tr = X[mask_tr], C[mask_tr]
X_va, C_va = X[mask_va], C[mask_va]
drv_va     = df_all.loc[mask_va, "Driver"].values

# ============================================================
# 5. SIGNET-V2 ARCHITECTURE (IMPROVED CAPACITY + STABILITY)
# ============================================================

class SIGNetV2(nn.Module):
    def __init__(self, x_dim, c_dim, z_dim=16):
        super().__init__()

        self.encoder = nn.Sequential(
            nn.Linear(x_dim + c_dim, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU()
        )
        self.mu = nn.Linear(128, z_dim)
        self.lv = nn.Linear(128, z_dim)

        self.decoder = nn.Sequential(
            nn.Linear(z_dim + c_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, x_dim)
        )

    def encode(self, x, c):
        h = self.encoder(torch.cat([x, c], 1))
        return self.mu(h), self.lv(h)

    def reparam(self, mu, lv):
        return mu + torch.randn_like(mu) * torch.exp(0.5 * lv)

    def decode(self, z, c):
        return self.decoder(torch.cat([z, c], 1))

    def forward(self, x, c):
        mu, lv = self.encode(x, c)
        z = self.reparam(mu, lv)
        return self.decode(z, c), mu, lv

def vae_loss(xr, x, mu, lv, beta=1.0):
    rec = F.mse_loss(xr, x)
    kl  = -0.5 * torch.mean(1 + lv - mu**2 - lv.exp())
    return rec + beta * kl, rec, kl

# ============================================================
# 6. TRAIN SIGNET-V2 (EARLY STOPPING + TIMING)
# ============================================================

def train_signet_v2(lr):
    model = SIGNetV2(X.shape[1], C.shape[1]).to(device)
    opt = optim.Adam(model.parameters(), lr=lr)

    hist = {"train": [], "val": [], "rec": [], "kl": []}
    best, patience = np.inf, 0

    t0 = time.time()

    for ep in range(1, 501):
        model.train()
        opt.zero_grad()

        xr, mu, lv = model(X_tr, C_tr)
        loss, rec, kl = vae_loss(xr, X_tr, mu, lv, beta=0.5)
        loss.backward()
        opt.step()

        model.eval()
        with torch.no_grad():
            xv, mu_v, lv_v = model(X_va, C_va)
            vloss, _, _ = vae_loss(xv, X_va, mu_v, lv_v, beta=0.5)

        hist["train"].append(loss.item())
        hist["val"].append(vloss.item())
        hist["rec"].append(rec.item())
        hist["kl"].append(kl.item())

        if vloss < best - 1e-4:
            best = vloss
            best_wts = copy.deepcopy(model.state_dict())
            patience = 0
        else:
            patience += 1

        if patience >= 30:
            break

    train_time = time.time() - t0
    model.load_state_dict(best_wts)
    return model, hist, train_time

model_v2, hist_v2, signet_train_time = train_signet_v2(1e-3)
torch.save(model_v2.state_dict(), f"{DIRS['mdl']}/signet_v2.pt")

# ============================================================
# 7. LATENT EXTRACTION
# ============================================================

model_v2.eval()
with torch.no_grad():
    Z_va = model_v2.encode(X_va, C_va)[0].cpu().numpy()

# ============================================================
# 8. LATENT CONSISTENCY METRICS (IEEE EXTENDED)
# ============================================================

latent_rows = []
for d in DRIVERS:
    Zd = Z_va[drv_va == d]

    latent_rows.append({
        "Driver": d,
        "Cosine_Sim": cosine_similarity(Zd).mean(),
        "Mean_Euclid": np.mean([
            euclidean(Zd[i], Zd[j])
            for i in range(len(Zd)) for j in range(i+1, len(Zd))
        ]),
        "Var_Trace": np.trace(np.cov(Zd.T)),
        "Coeff_Var": np.std(Zd) / (np.mean(np.abs(Zd)) + 1e-8)
    })

latent_intra_df = pd.DataFrame(latent_rows)

sil = silhouette_score(Z_va, LabelEncoder().fit_transform(drv_va))
db  = davies_bouldin_score(Z_va, LabelEncoder().fit_transform(drv_va))

# ============================================================
# 9. IDINFERNET-V2 (REAL LATENT CLASSIFIER)
# ============================================================

class IDInferNetV2(nn.Module):
    def __init__(self, z_dim, n):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(z_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, n)
        )
    def forward(self, z): return self.net(z)

le = LabelEncoder()
y = le.fit_transform(drv_va)

Ztr, Zte, ytr, yte = train_test_split(
    Z_va, y, stratify=y, test_size=0.3, random_state=SEED
)

Ztr = torch.tensor(Ztr, dtype=torch.float32).to(device)
Zte = torch.tensor(Zte, dtype=torch.float32).to(device)
ytr = torch.tensor(ytr, dtype=torch.long).to(device)
yte = torch.tensor(yte, dtype=torch.long).to(device)

idnet = IDInferNetV2(Ztr.shape[1], len(DRIVERS)).to(device)
opt = optim.Adam(idnet.parameters(), lr=1e-3)

acc_hist, loss_hist = [], []
t0 = time.time()

for ep in range(1, 301):
    opt.zero_grad()
    loss = F.cross_entropy(idnet(Ztr), ytr)
    loss.backward()
    opt.step()

    with torch.no_grad():
        acc = accuracy_score(yte.cpu(), idnet(Zte).argmax(1).cpu())
    acc_hist.append(acc)
    loss_hist.append(loss.item())

idinfer_train_time = time.time() - t0
torch.save(idnet.state_dict(), f"{DIRS['mdl']}/idinfernet_v2.pt")

# ============================================================
# 9A. INTER-DRIVER LATENT SEPARABILITY METRICS
# ============================================================

centroids = {}
for d in DRIVERS:
    centroids[d] = Z_va[drv_va == d].mean(axis=0)

inter_rows = []
for i, d1 in enumerate(DRIVERS):
    for d2 in DRIVERS[i+1:]:
        c1, c2 = centroids[d1], centroids[d2]
        inter_rows.append({
            "Pair": f"{d1}-{d2}",
            "Centroid_Euclidean": euclidean(c1, c2),
            "Centroid_Cosine_Distance": 1 - cosine_similarity([c1],[c2])[0,0],
            "Fisher_Ratio":
                np.linalg.norm(c1-c2)**2 /
                (np.var(Z_va[drv_va==d1]) + np.var(Z_va[drv_va==d2]) + 1e-8)
        })

latent_inter_df = pd.DataFrame(inter_rows)

# ============================================================
# 9B. LATENT SPACE VISUALIZATION
# ============================================================

pca = PCA(n_components=2)
Z_pca = pca.fit_transform(Z_va)

tsne = TSNE(n_components=2, perplexity=30, random_state=SEED)
Z_tsne = tsne.fit_transform(Z_va)

plt.figure(figsize=(6,5))
sns.scatterplot(x=Z_pca[:,0], y=Z_pca[:,1], hue=drv_va)
plt.title("SIGNet-v2 Latent Space (PCA)")
plt.tight_layout()
plt.savefig(f"{DIRS['fig']}/latent_pca.png", dpi=300)
plt.close()

plt.figure(figsize=(6,5))
sns.scatterplot(x=Z_tsne[:,0], y=Z_tsne[:,1], hue=drv_va)
plt.title("SIGNet-v2 Latent Space (t-SNE)")
plt.tight_layout()
plt.savefig(f"{DIRS['fig']}/latent_tsne.png", dpi=300)
plt.close()

# ============================================================
# 9C. EDGE DEPLOYMENT METRICS
# ============================================================

def latency_ms(model):
    model.eval()
    with torch.no_grad():
        t0 = time.time()
        for _ in range(100):
            _ = model(X_va[:1], C_va[:1])
    return (time.time() - t0)/100 * 1000

latency = latency_ms(model_v2)
flops, params = profile(model_v2, inputs=(X_va[:1], C_va[:1]), verbose=False)

edge_df = pd.DataFrame([{
    "Latency_ms": latency,
    "FLOPs_M": flops/1e6,
    "Params_M": params/1e6
}])

# ============================================================
# 10. DT-GDIN-V2 (DIGITAL-TWIN GENERATED CLASSIFIER)
# ============================================================

signatures = {}
for d in DRIVERS:
    idx = df_all["Driver"] == d
    with torch.no_grad():
        mu, _ = model_v2.encode(X[idx], C[idx])
        signatures[d] = mu.mean(0)

Xs, ys = [], []
for d in DRIVERS:
    mu = signatures[d]
    Cd = C[df_all["Driver"] == d]
    for _ in range(2500):
        z = mu + 0.7 * torch.randn_like(mu)
        c = Cd[np.random.randint(len(Cd))]
        with torch.no_grad():
            xg = model_v2.decode(z.unsqueeze(0), c.unsqueeze(0))
        Xs.append(xg.cpu().numpy()[0])
        ys.append(d)

Xs = np.array(Xs)
ys = le.fit_transform(ys)

Xtr, Xva, ytr, yva = train_test_split(
    Xs, ys, stratify=ys, test_size=0.3, random_state=SEED
)

class DTGDINV2(nn.Module):
    def __init__(self, d, n):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(d, 256),
            nn.ReLU(),
            nn.Linear(256, n)
        )
    def forward(self, x): return self.net(x)

dtgdin = DTGDINV2(Xtr.shape[1], len(DRIVERS)).to(device)
opt = optim.Adam(dtgdin.parameters(), lr=1e-3)

Xt = torch.tensor(Xtr, dtype=torch.float32).to(device)
yt = torch.tensor(ytr, dtype=torch.long).to(device)
Xv = torch.tensor(Xva, dtype=torch.float32).to(device)
yv = torch.tensor(yva, dtype=torch.long).to(device)

dt_acc = []
t0 = time.time()

for ep in range(1, 251):
    opt.zero_grad()
    loss = F.cross_entropy(dtgdin(Xt), yt)
    loss.backward()
    opt.step()

    with torch.no_grad():
        acc = accuracy_score(yv.cpu(), dtgdin(Xv).argmax(1).cpu())
    dt_acc.append(acc)

dtgdin_train_time = time.time() - t0
torch.save(dtgdin.state_dict(), f"{DIRS['mdl']}/dtgdin_v2.pt")

# ============================================================
# 10A. IDInferNet-v2 FULL EVALUATION
# ============================================================

y_pred = idnet(Zte).argmax(1).cpu().numpy()
y_prob = F.softmax(idnet(Zte), dim=1).detach().cpu().numpy()


cm_id = confusion_matrix(yte.cpu(), y_pred)

plt.figure(figsize=(5,4))
sns.heatmap(cm_id, annot=True, fmt="d",
            xticklabels=le.classes_,
            yticklabels=le.classes_)
plt.title("IDInferNet-v2 Confusion Matrix")
plt.tight_layout()
plt.savefig(f"{DIRS['fig']}/idinfernet_v2_cm.png", dpi=300)
plt.close()

# ROC
from sklearn.preprocessing import label_binarize
y_bin = label_binarize(yte.cpu(), classes=range(len(DRIVERS)))

roc_rows = []
plt.figure(figsize=(6,5))
for i, cls in enumerate(le.classes_):
    fpr, tpr, _ = roc_curve(y_bin[:,i], y_prob[:,i])
    auc_i = auc(fpr, tpr)
    roc_rows.append({"Driver": cls, "AUC": auc_i})
    plt.plot(fpr, tpr, label=f"{cls} (AUC={auc_i:.3f})")

plt.plot([0,1],[0,1],'k--')
plt.legend(); plt.title("IDInferNet-v2 ROC")
plt.tight_layout()
plt.savefig(f"{DIRS['fig']}/idinfernet_v2_roc.png", dpi=300)
plt.close()

id_report_df = pd.DataFrame(
    classification_report(yte.cpu(), y_pred, target_names=le.classes_, output_dict=True)
).T

# ============================================================
# 10B. DT-GDIN-v2 FULL EVALUATION
# ============================================================

dt_pred = dtgdin(Xv).argmax(1).cpu().numpy()
cm_dt = confusion_matrix(yv.cpu(), dt_pred)

plt.figure(figsize=(5,4))
sns.heatmap(cm_dt, annot=True, fmt="d",
            xticklabels=le.classes_,
            yticklabels=le.classes_)
plt.title("DT-GDIN-v2 Confusion Matrix")
plt.tight_layout()
plt.savefig(f"{DIRS['fig']}/dtgdin_v2_cm.png", dpi=300)
plt.close()

dt_report_df = pd.DataFrame(
    classification_report(yv.cpu(), dt_pred, target_names=le.classes_, output_dict=True)
).T

# ============================================================
# 11. SAVE FIGURES
# ============================================================

plt.figure()
plt.plot(hist_v2["train"], label="Train")
plt.plot(hist_v2["val"], label="Val")
plt.legend(); plt.title("SIGNet-v2 Loss")
plt.savefig(f"{DIRS['fig']}/signet_v2_loss.png", dpi=300)
plt.close()

# ============================================================
# 12. SAVE MASTER EXCEL
# ============================================================

excel_path = f"{DIRS['xls']}/SIGNetV2_MasterResults.xlsx"
with pd.ExcelWriter(excel_path, engine="openpyxl") as w:

    # SIGNet
    pd.DataFrame(hist_v2).to_excel(w, "SIGNetV2_Loss", index=False)
    latent_intra_df.to_excel(w, "Latent_Intra", index=False)
    latent_inter_df.to_excel(w, "Latent_Inter", index=False)

    # Cluster quality
    pd.DataFrame({
        "Silhouette": [sil],
        "Davies_Bouldin": [db]
    }).to_excel(w, "Cluster_Quality", index=False)

    # IDInferNet
    pd.DataFrame({"Accuracy": acc_hist, "Loss": loss_hist}).to_excel(w, "IDInferNet_Ablation", index=False)
    id_report_df.to_excel(w, "IDInferNet_Report")
    pd.DataFrame(cm_id).to_excel(w, "IDInferNet_Confusion")

    # DT-GDIN
    pd.DataFrame(dt_acc, columns=["Accuracy"]).to_excel(w, "DTGDIN_Ablation", index=False)
    dt_report_df.to_excel(w, "DTGDIN_Report")
    pd.DataFrame(cm_dt).to_excel(w, "DTGDIN_Confusion")

    # Edge + Timing
    edge_df.to_excel(w, "Edge_Metrics", index=False)
    pd.DataFrame([{
        "SIGNetV2_Train_s": signet_train_time,
        "IDInferNetV2_Train_s": idinfer_train_time,
        "DTGDIN_V2_Train_s": dtgdin_train_time
    }]).to_excel(w, "Timing", index=False)

print("✅ SIGNet-v2 NOTEBOOK COMPLETE")
print(f"📁 Results saved to: {RES_ROOT}")


⚙️ Using device: cuda
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ SIGNet-v2 NOTEBOOK COMPLETE
📁 Results saved to: /content/drive/MyDrive/DT_Driver_Wise_Data/SIGNetV2_Results


/tmp/ipython-input-3494391446.py:499: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  pd.DataFrame(hist_v2).to_excel(w, "SIGNetV2_Loss", index=False)
/tmp/ipython-input-3494391446.py:500: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  latent_intra_df.to_excel(w, "Latent_Intra", index=False)
/tmp/ipython-input-3494391446.py:501: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  latent_inter_df.to_excel(w, "Latent_Inter", index=False)
/tmp/ipython-input-3494391446.py:507: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  }).to_excel(w, "Cluster_Quality", index=False)
/tmp/ipython-input-3494391446.py:510: FutureWarning: Starting with pandas v

In [7]:
# ============================================================
# CVAE_Digital_Twin-v2 | MASTER OPTIMIZATION STAGE (FINAL)
# FP32 + Structured Pruning + INT8 Quantization
# ARCHITECTURES & DATA PIPELINE EXACTLY PRESERVED
# SELF-CONTAINED | REPRODUCIBLE | IEEE-GRADE
# ============================================================

!pip -q install thop openpyxl

# ============================================================
# 0. IMPORTS & GLOBALS
# ============================================================

import os, time, copy
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.utils.prune as prune

from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from thop import profile
from google.colab import drive

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# ============================================================
# 1. PATHS & CONSTANTS
# ============================================================

drive.mount("/content/drive")

DATA_ROOT = "/content/drive/MyDrive/DT_Driver_Wise_Data"
RES_ROOT  = f"{DATA_ROOT}/SIGNetV2_Results"

DIRS = {
    "mdl": f"{RES_ROOT}/Models",
    "xls": f"{RES_ROOT}/Excel",
}
for d in DIRS.values():
    os.makedirs(d, exist_ok=True)

DRIVERS = ["B", "D", "F"]
FEATURES = [
    "Long_Term_Fuel_Trim_Bank1",
    "Engine_coolant_temperature.1",
    "Activation_of_Air_compressor",
    "Torque_of_friction",
    "Engine_soacking_time",
    "Intake_air_pressure",
]
TIME_COL = "Time(s)"

# ============================================================
# 2. DATA LOADING (ROBUST, FAIL-FAST)
# ============================================================

dfs = []
missing = []

for d in DRIVERS:
    for s in ["Train", "Valid"]:
        p = f"{DATA_ROOT}/Driver_{d}_{s}.csv"
        if os.path.exists(p):
            df = pd.read_csv(p)
            df["Driver"] = d
            df["Split"] = s
            dfs.append(df)
        else:
            missing.append(p)

if len(dfs) == 0:
    raise RuntimeError(
        "No CSV files found. Checked paths:\n" + "\n".join(missing)
    )

df_all = pd.concat(dfs, ignore_index=True)
df_all = df_all.dropna(subset=FEATURES + [TIME_COL])

print(f"Loaded {len(df_all)} samples")

# ============================================================
# 3. PREPROCESSING (BIT-IDENTICAL TO TRAINING)
# ============================================================

sc_x = StandardScaler()
sc_t = StandardScaler()
enc  = OneHotEncoder(sparse_output=False)
le   = LabelEncoder()

X = sc_x.fit_transform(df_all[FEATURES])
T = sc_t.fit_transform(df_all[[TIME_COL]])
D = enc.fit_transform(df_all[["Driver"]])
C = np.concatenate([T, D], axis=1)

X = torch.tensor(X, dtype=torch.float32)
C = torch.tensor(C, dtype=torch.float32)

mask_va = df_all["Split"] == "Valid"
X_va = X[mask_va].to(device)
C_va = C[mask_va].to(device)
drv_va = df_all.loc[mask_va, "Driver"].values

# ============================================================
# 4. MODEL ARCHITECTURES (EXACT, UNCHANGED)
# ============================================================

class SIGNetV2(nn.Module):
    def __init__(self, x_dim, c_dim, z_dim=16):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(x_dim + c_dim, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU()
        )
        self.mu = nn.Linear(128, z_dim)
        self.lv = nn.Linear(128, z_dim)
        self.decoder = nn.Sequential(
            nn.Linear(z_dim + c_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, x_dim)
        )

    def encode(self, x, c):
        h = self.encoder(torch.cat([x, c], 1))
        return self.mu(h), self.lv(h)

    def reparam(self, mu, lv):
        return mu + torch.randn_like(mu) * torch.exp(0.5 * lv)

    def decode(self, z, c):
        return self.decoder(torch.cat([z, c], 1))

    def forward(self, x, c):
        mu, lv = self.encode(x, c)
        z = self.reparam(mu, lv)
        return self.decode(z, c), mu, lv


class IDInferNetV2(nn.Module):
    def __init__(self, z_dim, n):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(z_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, n)
        )
    def forward(self, z): return self.net(z)


class DTGDINV2(nn.Module):
    def __init__(self, d, n):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(d, 256),
            nn.ReLU(),
            nn.Linear(256, n)
        )
    def forward(self, x): return self.net(x)

# ============================================================
# 5. LOAD TRAINED WEIGHTS
# ============================================================

x_dim = X_va.shape[1]
c_dim = C_va.shape[1]
z_dim = 16
n_cls = len(DRIVERS)

signet = SIGNetV2(x_dim, c_dim).to(device)
signet.load_state_dict(torch.load(f"{DIRS['mdl']}/signet_v2.pt"))
signet.eval()

with torch.no_grad():
    Z_va = signet.encode(X_va, C_va)[0]

idnet = IDInferNetV2(z_dim, n_cls).to(device)
idnet.load_state_dict(torch.load(f"{DIRS['mdl']}/idinfernet_v2.pt"))
idnet.eval()

dtgdin = DTGDINV2(x_dim, n_cls).to(device)
dtgdin.load_state_dict(torch.load(f"{DIRS['mdl']}/dtgdin_v2.pt"))
dtgdin.eval()

# ============================================================
# 6. EDGE PROFILING (QUANT-SAFE)
# ============================================================

def edge_profile(model, inputs, dtype_bytes, runs=50):
    model.eval()
    try:
        dev = next(model.parameters()).device
    except StopIteration:
        dev = torch.device("cpu")

    if isinstance(inputs, tuple):
        inputs = tuple(i.to(dev) for i in inputs)
    else:
        inputs = inputs.to(dev)

    with torch.no_grad():
        flops, params = profile(model, inputs=inputs, verbose=False)
        t0 = time.time()
        for _ in range(runs):
            _ = model(*inputs) if isinstance(inputs, tuple) else model(inputs)
        latency = (time.time() - t0) / runs * 1000

    return {
        "Params": int(params),
        "FLOPs": int(flops),
        "Latency_ms": latency,
        "Memory_KB": params * dtype_bytes / 1024,
        "Energy_mJ_est": flops * 3e-9
    }

# ============================================================
# 7. OPTIMIZATION OPERATORS
# ============================================================

def prune_model(model, amount=0.3):
    m = copy.deepcopy(model)
    for l in m.modules():
        if isinstance(l, nn.Linear):
            prune.ln_structured(l, "weight", amount, n=1, dim=0)
            prune.remove(l, "weight")
    return m

def quant_model(model):
    return torch.quantization.quantize_dynamic(
        model.cpu(), {nn.Linear}, dtype=torch.qint8
    )

# ============================================================
# 8. OPTIMIZED MODELS
# ============================================================

models = {
    "SIGNetV2_FP32": signet,
    "SIGNetV2_Pruned": prune_model(signet),
    "SIGNetV2_Quant": quant_model(signet),

    "IDInferNetV2_FP32": idnet,
    "IDInferNetV2_Pruned": prune_model(idnet),
    "IDInferNetV2_Quant": quant_model(idnet),

    "DTGDINV2_FP32": dtgdin,
    "DTGDINV2_Pruned": prune_model(dtgdin),
    "DTGDINV2_Quant": quant_model(dtgdin),
}

# ============================================================
# 9. EDGE METRICS (REAL DATA)
# ============================================================

rows = []

for name, model in models.items():
    is_quant = "Quant" in name
    dtype_bytes = 1 if is_quant else 4

    if "SIGNetV2" in name:
        inputs = (X_va[:1], C_va[:1])
    elif "IDInferNetV2" in name:
        inputs = (Z_va[:1],)
    else:
        inputs = (X_va[:1],)

    rows.append({
        "Model": name,
        **edge_profile(model, inputs, dtype_bytes)
    })

    torch.save(model.state_dict(), f"{DIRS['mdl']}/{name}.pt")

edge_df = pd.DataFrame(rows)

# ============================================================
# 10. SAVE RESULTS
# ============================================================

out_xls = f"{DIRS['xls']}/SIGNetV2_Edge_Optimization.xlsx"
edge_df.to_excel(out_xls, index=False)

print("MASTER OPTIMIZATION COMPLETE")
print("Models:", DIRS["mdl"])
print("Excel:", out_xls)


Device: cuda
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loaded 95368 samples


/tmp/ipython-input-1412723234.py:240: DeprecationWarning: torch.ao.quantization is deprecated and will be removed in 2.10. 
For migrations of users: 
1. Eager mode quantization (torch.ao.quantization.quantize, torch.ao.quantization.quantize_dynamic), please migrate to use torchao eager mode quantize_ API instead 
2. FX graph mode quantization (torch.ao.quantization.quantize_fx.prepare_fx,torch.ao.quantization.quantize_fx.convert_fx, please migrate to use torchao pt2e quantization API instead (prepare_pt2e, convert_pt2e) 
3. pt2e quantization has been migrated to torchao (https://github.com/pytorch/ao/tree/main/torchao/quantization/pt2e) 
see https://github.com/pytorch/ao/issues/2259 for more details
  return torch.quantization.quantize_dynamic(


MASTER OPTIMIZATION COMPLETE
Models: /content/drive/MyDrive/DT_Driver_Wise_Data/SIGNetV2_Results/Models
Excel: /content/drive/MyDrive/DT_Driver_Wise_Data/SIGNetV2_Results/Excel/SIGNetV2_Edge_Optimization.xlsx
